<a href="https://colab.research.google.com/github/rklepov/hse-cs-ml-2018-2019/blob/master/08-spark/02-sql/spark_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[***Lecture 2 slides***](https://hackmd.io/@J_qqq0PjTGK1be0341GpYA/SJZDs66fm#/ "Как Spark устроен внутри &amp; SparkSQL - HackMD"): https://hackmd.io/@J_qqq0PjTGK1be0341GpYA/SJZDs66fm#/

---

*   [Introduction to Distributed File System – NFS Vs. HDFS](https://diveintohadoop.wordpress.com/2015/04/04/introduction-to-distributed-file-system-nfs-vs-hdfs/ "Introduction to Distributed File System &#8211; NFS Vs. HDFS | All About hadoop")
*   [Apache Parquet](https://parquet.apache.org/ "Apache Parquet")
*   [Predicate Pushdown](https://medium.com/microsoftazure/data-at-scale-learn-how-predicate-pushdown-will-save-you-money-7063b80878d7 "Data at scale - Learn how Predicate Pushdown will save you money")
*   [pyspark.sql module](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html "pyspark.sql module &#8212; PySpark master documentation")

In [1]:
!pip search spark | grep INSTALLED || pip install pyspark==2.4.0 findspark

  INSTALLED: 2.4.0


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre/"

import findspark
findspark.init('/usr/local/lib/python3.6/dist-packages/pyspark/')

import pyspark

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
from zipfile import ZipFile
from io import BytesIO
import urllib.request

ZipFile.extract(
    ZipFile(
        BytesIO(
            urllib
            .request
            .urlopen('http://s3.amazonaws.com/alexa-static/top-1m.csv.zip')
            .read()
        )
    ),
    'top-1m.csv'
)

'/content/top-1m.csv'

In [0]:
spark =pyspark.sql.SparkSession.builder.getOrCreate()

In [0]:
alexa = spark.read.csv('top-1m.csv')

In [7]:
alexa.show()

+---+-------------+
|_c0|          _c1|
+---+-------------+
|  1|   google.com|
|  2|  youtube.com|
|  3| facebook.com|
|  4|    baidu.com|
|  5|wikipedia.org|
|  6|       qq.com|
|  7|   taobao.com|
|  8|    tmall.com|
|  9|   amazon.com|
| 10|    yahoo.com|
| 11|       jd.com|
| 12|  twitter.com|
| 13|     sohu.com|
| 14|     live.com|
| 15|instagram.com|
| 16|    yandex.ru|
| 17|  sina.com.cn|
| 18|    weibo.com|
| 19|   reddit.com|
| 20| linkedin.com|
+---+-------------+
only showing top 20 rows



In [0]:
from pyspark.sql import types

schema = types.StructType().add('rank', 'integer').add('site', 'string')

real_alexa = spark.read.csv('top-1m.csv', schema=schema)

In [9]:
real_alexa.show()

+----+-------------+
|rank|         site|
+----+-------------+
|   1|   google.com|
|   2|  youtube.com|
|   3| facebook.com|
|   4|    baidu.com|
|   5|wikipedia.org|
|   6|       qq.com|
|   7|   taobao.com|
|   8|    tmall.com|
|   9|   amazon.com|
|  10|    yahoo.com|
|  11|       jd.com|
|  12|  twitter.com|
|  13|     sohu.com|
|  14|     live.com|
|  15|instagram.com|
|  16|    yandex.ru|
|  17|  sina.com.cn|
|  18|    weibo.com|
|  19|   reddit.com|
|  20| linkedin.com|
+----+-------------+
only showing top 20 rows



In [10]:
real_alexa.orderBy('rank').show()

+----+-------------+
|rank|         site|
+----+-------------+
|   1|   google.com|
|   2|  youtube.com|
|   3| facebook.com|
|   4|    baidu.com|
|   5|wikipedia.org|
|   6|       qq.com|
|   7|   taobao.com|
|   8|    tmall.com|
|   9|   amazon.com|
|  10|    yahoo.com|
|  11|       jd.com|
|  12|  twitter.com|
|  13|     sohu.com|
|  14|     live.com|
|  15|instagram.com|
|  16|    yandex.ru|
|  17|  sina.com.cn|
|  18|    weibo.com|
|  19|   reddit.com|
|  20| linkedin.com|
+----+-------------+
only showing top 20 rows



In [11]:
real_alexa.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- site: string (nullable = true)



In [0]:
from pyspark.sql import functions

In [13]:
(
    real_alexa
    .select(
        functions.split('site', '\.').alias('token')
    )
    .show()
)

+----------------+
|           token|
+----------------+
|   [google, com]|
|  [youtube, com]|
| [facebook, com]|
|    [baidu, com]|
|[wikipedia, org]|
|       [qq, com]|
|   [taobao, com]|
|    [tmall, com]|
|   [amazon, com]|
|    [yahoo, com]|
|       [jd, com]|
|  [twitter, com]|
|     [sohu, com]|
|     [live, com]|
|[instagram, com]|
|    [yandex, ru]|
| [sina, com, cn]|
|    [weibo, com]|
|   [reddit, com]|
| [linkedin, com]|
+----------------+
only showing top 20 rows



In [14]:
(
    real_alexa
    .select(
        functions.explode(
            functions.split('site', '\.')
        ).alias('token')
    )
    .groupBy('token')
    .count()
    .withColumnRenamed('count', 'shmount')
    .orderBy(functions.desc('shmount'))
    .show()
)

+--------+-------+
|   token|shmount|
+--------+-------+
|     com| 550206|
|     org|  55565|
|     net|  41564|
|      ru|  37602|
|      co|  36202|
|      de|  26294|
|      br|  25347|
|      uk|  15774|
|      pl|  13845|
|blogspot|  13567|
|      in|  13148|
|      ir|  11629|
|      jp|  10878|
|      it|  10664|
|     gov|   9807|
|      fr|   9447|
|      au|   8929|
|     edu|   8303|
|    info|   7954|
|      kr|   7842|
+--------+-------+
only showing top 20 rows



In [15]:
(
    real_alexa
    .select(
        functions.explode(
            functions.split('site', '\.')
        ).alias('token'),
        functions.col('site')
    )
    .show()
)

+---------+-------------+
|    token|         site|
+---------+-------------+
|   google|   google.com|
|      com|   google.com|
|  youtube|  youtube.com|
|      com|  youtube.com|
| facebook| facebook.com|
|      com| facebook.com|
|    baidu|    baidu.com|
|      com|    baidu.com|
|wikipedia|wikipedia.org|
|      org|wikipedia.org|
|       qq|       qq.com|
|      com|       qq.com|
|   taobao|   taobao.com|
|      com|   taobao.com|
|    tmall|    tmall.com|
|      com|    tmall.com|
|   amazon|   amazon.com|
|      com|   amazon.com|
|    yahoo|    yahoo.com|
|      com|    yahoo.com|
+---------+-------------+
only showing top 20 rows



In [16]:
reverse_idx = (
    real_alexa
    .select(
        functions.explode(
            functions.split('site', '\.')
        ).alias('token'),
        functions.col('site')
    )
    .groupBy('token')
    .agg(functions.collect_set('site').alias('sites'))
)

reverse_idx.cache()

reverse_idx.show()

+--------------+--------------------+
|         token|               sites|
+--------------+--------------------+
|         03442|      [03442.com.ar]|
|      0rechner|       [0rechner.de]|
|      1000ordi|       [1000ordi.ch]|
|      100fenlm|       [100fenlm.cn]|
|  100x100banco|  [100x100banco.com]|
|     103apteka|      [103apteka.kz]|
|      10starhd|      [10starhd.com]|
|     10toptest|[10toptest.de, 10...|
|          11-2|           [11-2.ru]|
|       111kent|       [111kent.com]|
|         11888|          [11888.gr]|
|        11math|        ath.com]|
|123freevectors|[123freevectors.com]|
|  123movieshit|  ovieshit.com]|
|    123webimmo|    [123webimmo.com]|
|  1404boylston|  [1404boylston.com]|
|           14p|            [14p.ir]|
|      1500days|      [1500days.com]|
|        183mir|         ir.cn]|
|       18gifts|       [18gifts.com]|
+--------------+--------------------+
only showing top 20 rows



In [17]:
reverse_idx.where("token = 'google'").show()

+------+--------------------+
| token|               sites|
+------+--------------------+
|google|[safety.google, d...|
+------+--------------------+



In [18]:
(
    real_alexa
    .select(
        functions.explode(
            functions.split('site', '\.')
        ).alias('token'),
        functions.col('site'),
        functions.col('rank')
    )
    .groupBy('token')
    .agg(
        {
            'site' : 'collect_set',
            'rank' : 'max'
        }
    )
    .show()
)

+--------------+--------------------+---------+
|         token|   collect_set(site)|max(rank)|
+--------------+--------------------+---------+
|         03442|      [03442.com.ar]|   457356|
|      0rechner|       [0rechner.de]|   648664|
|      1000ordi|       [1000ordi.ch]|   700985|
|      100fenlm|       [100fenlm.cn]|   744445|
|  100x100banco|  [100x100banco.com]|    35564|
|     103apteka|      [103apteka.kz]|   436524|
|      10starhd|      [10starhd.com]|   141444|
|     10toptest|[10toptest.de, 10...|   325930|
|          11-2|           [11-2.ru]|   480310|
|       111kent|       [111kent.com]|   744464|
|         11888|          [11888.gr]|    25317|
|        11math|        ath.com]|   152496|
|123freevectors|[123freevectors.com]|    92372|
|  123movieshit|  ovieshit.com]|   177149|
|    123webimmo|    [123webimmo.com]|   240937|
|  1404boylston|  [1404boylston.com]|   907529|
|           14p|            [14p.ir]|   612943|
|      1500days|      [1500days.com]|   608679|
|

In [19]:
real_alexa.toPandas().head()

,rank,site
0,1,google.com
1,2,youtube.com
2,3,facebook.com
3,4,baidu.com
4,5,wikipedia.org
